Need to have a TPU go to edit and then notebook settings and click TPU V2. It's a roulette hopefully you get one, just have to keep retrying to connect and eventually you get one

In [1]:
import jax
jax.distributed.initialize()

In [28]:
jax.device_count()

8

In [37]:
import jax.numpy as jnp
x = jnp.arange(164 * 48.).reshape((8, 984))
y = jnp.arange(48 * 164.).reshape((8, 984)) ** 2
out = jax.pmap(jnp.matmul)(x, y)
print(out)

[2.3390378e+11 3.5123650e+12 1.5225643e+13 4.0998869e+13 8.6457121e+13
 1.5722555e+14 2.5892930e+14 3.9719337e+14]


In [24]:
def add(x, y):
  return x + y
j = jax.jit(add).lower(2,2)
k = jax.jit(jnp.dot).lower(2,2)
print(k.as_text())

module @jit_dot attributes {mhlo.num_partitions = 1 : i32, mhlo.num_replicas = 1 : i32} {
  func.func public @main(%arg0: tensor<i32> {mhlo.layout_mode = "default"}, %arg1: tensor<i32> {mhlo.layout_mode = "default"}) -> (tensor<i32> {jax.result_info = "", mhlo.layout_mode = "default"}) {
    %0 = stablehlo.convert %arg0 : tensor<i32>
    %1 = stablehlo.convert %arg1 : tensor<i32>
    %2 = stablehlo.multiply %0, %1 : tensor<i32>
    %3 = stablehlo.convert %2 : tensor<i32>
    return %3 : tensor<i32>
  }
}



In [38]:
print(jax.jit(jax.pmap(jnp.matmul)).lower(x,y).as_text())

module @jit_matmul attributes {mhlo.num_partitions = 1 : i32, mhlo.num_replicas = 8 : i32} {
  func.func public @main(%arg0: tensor<8x984xf32> {mhlo.layout_mode = "default"}, %arg1: tensor<8x984xf32> {mhlo.layout_mode = "default"}) -> (tensor<8xf32> {jax.result_info = "", mhlo.layout_mode = "default"}) {
    %0 = stablehlo.constant dense<0> : tensor<ui32>
    %1 = stablehlo.constant dense<1> : tensor<ui32>
    %2 = stablehlo.constant dense<8> : tensor<ui32>
    %3 = stablehlo.replica_id : tensor<ui32>
    %4 = stablehlo.divide %3, %1 : tensor<ui32>
    %5 = stablehlo.remainder %4, %2 : tensor<ui32>
    %6 = stablehlo.dynamic_slice %arg0, %5, %0, sizes = [1, 984] : (tensor<8x984xf32>, tensor<ui32>, tensor<ui32>) -> tensor<1x984xf32>
    %7 = stablehlo.reshape %6 : (tensor<1x984xf32>) -> tensor<984xf32>
    %8 = stablehlo.constant dense<0> : tensor<ui32>
    %9 = stablehlo.constant dense<1> : tensor<ui32>
    %10 = stablehlo.constant dense<8> : tensor<ui32>
    %11 = stablehlo.replica_id

/usr/local/lib/python3.10/dist-packages/jax/_src/interpreters/pxla.py:1858: UserWarning: The jitted function matmul includes a pmap. Using jit-of-pmap can lead to inefficient data movement, as the outer jit does not preserve sharded data representations and instead collects input and output arrays onto a single device. Consider removing the outer jit unless you know what you're doing. See https://github.com/google/jax/issues/2926.
  warnings.warn(
